In [2]:
import pandas as pd

def add_space(x):
    if (' ' not in x)&(len(x)<=5):
        return ' ' + x + ' '
    else:
        return x

def product_name_postprocess(x):
    x = x.replace('-',' ')
    x = x.strip()
    x = add_space(x)
    return x

df = pd.read_excel('..\data\寶典\寶典.v5.20211006.xlsx',engine='openpyxl')
df = df.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df['品名'] = df['品名'].apply(lambda x:product_name_postprocess(x))
df['RIGID'].fillna('not value',inplace=True)
df['EXTN'].fillna('not value',inplace=True)
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))
NAME = 'POLYPROPYLENE RESIN GRADE'

In [3]:
品名2部門[NAME]

'塑聚丙烯部'

In [4]:
品名2代號[NAME]

'1P'

In [5]:
train_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')
train_df['Y_label'] = train_df['Y_label'].apply(lambda x:x.strip()) #針對SPEC去除左右空白
train_df[(train_df['Y_label']==NAME)]

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2


In [6]:
# 根據品名從訓練資料搜索EXPNO,然後把EXPNO代入保典裡找公司部門
def find_department(x):
    try:
        return df.loc[df['公司代號']==train_df.loc[train_df.Y_label==x,'EXPNO'].dropna().value_counts().sort_values().index[-1],'公司事業部門'].value_counts().sort_values().index[-1]
    except:
        return 'not from_pretrained'

In [7]:
find_department(NAME)

'not from_pretrained'

In [8]:
df[df['品名']==NAME]

,RIGID,EXTN,公司代號,公司事業部門,品名,memo
662,工讀生,6136,1P,塑聚丙烯部,POLYPROPYLENE RESIN GRADE,NaN


In [9]:
train_df[train_df['Y_label']==NAME]

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2


In [10]:
a = ' QUANTITY 155 MT OF PVC  SUSPENSION RESIN S70R AT USD 1620 PER MTAS PER TRICON REF NO 376031 DATED 16092021QUANTITY 150 MT OF PVC  SUSPENSION RESIN S75 AT USD 1640 PER MTAS PER TRICON REF NO 376032 DATED 16092021TERMS OF DELIVERY CIF PIPAVAV OR MUNDRA INDIA INCOTERMS 2020 '


In [11]:
def g(x):
    x = x.replace('   ', ' ')
    x = x.replace('  ', ' ')
    return x

In [12]:
a

' QUANTITY 155 MT OF PVC  SUSPENSION RESIN S70R AT USD 1620 PER MTAS PER TRICON REF NO 376031 DATED 16092021QUANTITY 150 MT OF PVC  SUSPENSION RESIN S75 AT USD 1640 PER MTAS PER TRICON REF NO 376032 DATED 16092021TERMS OF DELIVERY CIF PIPAVAV OR MUNDRA INDIA INCOTERMS 2020 '

In [13]:
g(a)

' QUANTITY 155 MT OF PVC SUSPENSION RESIN S70R AT USD 1620 PER MTAS PER TRICON REF NO 376031 DATED 16092021QUANTITY 150 MT OF PVC SUSPENSION RESIN S75 AT USD 1640 PER MTAS PER TRICON REF NO 376032 DATED 16092021TERMS OF DELIVERY CIF PIPAVAV OR MUNDRA INDIA INCOTERMS 2020 '